<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/banner.png)

# <h1><center>Tarea Tutorial - Semana 1 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/PruebaBanner.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>Aproximación de Funciones</h1></center>

En este tutorial aprenderás acerca de las ventajas que tiene utilizar aproximación de funciones en problemas de aprendizaje por refuerzo profundo. Ilustraremos el proceso con el problema de <a href="https://gymnasium.farama.org/environments/classic_control/cart_pole/">Cart Pole</a>, incluido en las librerías de Gym, e implementaremos redes nuronales con el framework <a href="https://github.com/inarikami/keras-rl2/blob/master/examples/dqn_cartpole.py">keras-rl2</a>, que ya incluye varios algoritmos de aprendizaje por refuerzo con tensorflow. El objetivo principal será comparar el desempeño de distintos algoritmos en su versión tabular y su versión con redes neuronales. Este notebook tutorial se divide en las siguientes secciones:

<h1 id="indice">Tabla de Contenidos</h1>
<ul>
  <li><a href="#objetivos-de-aprendizaje">1. Objetivos de Aprendizaje</a></li>
  <li><a href="#marco-teorico">2. Marco Teórico</a></li>
  <li><a href="#instalacion-de-librerias">3. Instalación de Librerías</a></li>
  <li><a href="#familiarizacion-con-gym">4. Familiarización con el Entorno de Gym</a></li>
  <li><a href="#metodos-tabulares">5. Métodos Tabulares</a></li>
  <li><a href="#metodos-redes-neuronales">6. Métodos con Redes Neuronales</a></li>
  <li><a href="#comparacion">7. Comparación</a></li>
  <li><a href="#conclusiones">8. Conclusiones</a></li>
</ul>


<h2 id="objetivos-de-aprendizaje">1. Objetivos de Aprendizaje</h2>
<p>Contenido de la sección 1.</p>



<h2 id="marco-teorico">2. Marco Teórico</h2>
<p>Contenido de la sección 2.</p>



<h2 id="instalacion-de-librerias">3. Instalación de Librerías</h2>
<p>Contenido de la sección 3.</p>

<h2 id="familiarizacion-con-gym">4. Familiarización con el Entorno de Gym</h2>
<p>Contenido de la sección 4.</p>

<h2 id="metodos-tabulares">5. Métodos Tabulares</h2>
<p>Contenido de la sección 5.</p>

<h2 id="metodos-redes-neuronales">6. Métodos con Redes Neuronales</h2>
<p>Contenido de la sección 6.</p>

<h2 id="comparacion">7. Comparación</h2>
<p>Contenido de la sección 7.</p>

<h2 id="conclusiones">8. Conclusiones</h2>
<p>Contenido de la sección 8.</p>